# Deep Learning: Hands-on Laboratory

Here we will use **PyTorch** to build real neural networks.
1.  **Tensors & Gradients**: Understanding the building blocks.
2.  **The MNIST Project**: Building a brain to read handwritten digits.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

## Part 1: Autograd (The Magic of Calculus)

We will prove that PyTorch can calculate derivatives for us automatically.
Equation: $y = x^2 + 3x + 5$. Therefore, Derivative $y' = 2x + 3$.

In [1]:
x = torch.tensor(2.0, requires_grad=True)
y = x**2 + 3*x + 5

y.backward() # Calculates gradient

print(f"Value of x: {x.item()}")
print(f"Value of y: {y.item()}")
print(f"Calculated Gradient (dy/dx): {x.grad.item()}")
print(f"Expected Gradient (2x + 3): {2*2 + 3}")

NameError: name 'torch' is not defined

## Part 2: Building a Neural Network (MNIST Classifier)

We will build a Multi-Layer Perceptron (MLP) to recognize digits 0-9.

In [ ]:
# 1. Prepare Data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# 2. Define Network Architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128) # Input Layer (784 pixels)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)    # Hidden Layer
        self.fc3 = nn.Linear(64, 10)     # Output Layer (10 digits)

    def forward(self, x):
        x = x.view(-1, 28*28) # Flatten the image
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)

In [ ]:
# 3. Training Loop
epochs = 3
loss_history = []

print("Starting Training...")
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    loss_history.append(avg_loss)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

# Visualize Loss
plt.plot(loss_history)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
# 4. Evaluation
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on Test Set: {100 * correct / total:.2f}%")